## Init 🍄

---

### Dependencies:

In [ ]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [ ]:
# edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
edge.maximize_window()

### Functions:

In [ ]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [ ]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [ ]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [ ]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [ ]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [ ]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [ ]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [ ]:
def get_len(features:list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
             lengths.append(len(feature))
    return lengths

In [ ]:
def check_len(lengths:list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

## Extraction 🧲

----

### Sedan:

Variables pool.

In [ ]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 1

In [ ]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [ ]:
edge.get(sedan["url"])

In [ ]:
# close_prompt(edge)

In [ ]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [ ]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [ ]:
prices = extract_feature(edge, sedan["price"])
models = extract_feature(edge, sedan["model"])

Extract car year, engine & kilometers.

In [ ]:
years = extract_feature(edge, sedan["year"])
engines = extract_feature(edge, sedan["engine"])
kilometers = extract_feature(edge, sedan["kilometers"])

In [ ]:
lengths = [prices, models, years, engines, kilometers, descriptions]
get_len(lengths)

## Cleaning 🤿

### Functions:

In [ ]:
def parse_price(dataframe: pd.DataFrame, feature:str, pattern:str) -> None:
    """
    Parse financial features like prices. Works with $#,### or $#,###\n* pattern
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe [feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


In [ ]:
def remove_chars(dataframe: pd.DataFrame, feature:str, wildcards:str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        ,axis = 1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [ ]:
def replace_datapoint(value:str, replacement:any, datalist: list[any]) -> None:
    """
    Replace a datapoint if we know the value of datapoint to replace. Useful for kilometers list bug.
    :param value: value of datapoint to remove
    :param replacement: value of datapoint to insert
    :param datalist: list where the operation will be applied
    """
    try:
        to_replace = datalist.index(value)
        datalist[to_replace] = replacement
    except ValueError as ve:
        print("That values doesn't exists")

In [ ]:
PRICE_RG: str = r"^\$(?P<prices>\d+,\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG:str = '(\n|\r)+'

---

Remove noise elements from data list

In [ ]:
# engines.append("Gasolina")

In [ ]:
years.remove("Ver más")

In [ ]:
models.remove("Blue Book Encuentra24")

In [ ]:
# kilometers.remove("Gasolina")
# kilometers.insert(-2, "0")

In [ ]:
# # engines.pop(-1)

Create dataframe to process data extracted

In [ ]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices),len(descriptions)]
lengths

This is necessary when exists an announcement with no description or engine. Basically to append missing values.

In [ ]:
# descriptions.insert(-15, "No description") (exact position, value)

In [ ]:
# descriptions.append("No description")

In [ ]:
kilometers.index("108000 km")
kilometers[5:9]

In [ ]:
if check_len(lengths):

    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    # Export dataframe in .csv format
    persistence(df_temp)


Messy engines datapoints

In [135]:
df_temp.head()

,model,year,kilometers,engine,price,description
0,Kia Forte,2016,34000,Gasolina,8500,Financiamiento: si DESCRIPCIÓN CALIFICACIONES ...
1,Nissan Versa,2017,78000,Gasolina,7200,Color : Azul DESCRIPCIÓN CALIFICACIONES SEGURI...
2,Toyota Corolla,2018,39000,Gasolina,11500,Precio negociable: si Color : Negro perlado CO...
3,Mitsubishi Lancer,2017,68000,Gasolina,8700,Color : Azul Vendo Mitsubishi Lancer 2017 Full...
4,Mazda 3,2010,157255,Gasolina,5800,Financiamiento: no Precio negociable: si Color...


In [ ]:
# df_temp["engine"].iloc[31] = "Híbrido"

In [134]:
df_temp.query("engine == 'Híbrido'")

,model,year,kilometers,engine,price,description
31,Toyota Corolla,2009,195504,Híbrido,17000,Garantía: Como es visto no hay garantía Precio...


In [136]:
persistence(df_temp)